In [1]:
import pandas as pd
import numpy as np
import function as fn

# feature extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity




In [2]:
df_movie = pd.read_csv("/Users/nontanatto/Desktop/movie rec/ml-25m/movies.csv")
df_movie

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


Cleaning 

In [3]:
df_movie['cleaned title'] = df_movie['title'].apply(fn.clean_title)

In [4]:
df_movie

,movieId,title,genres,cleaned title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II
...,...,...,...,...
62418,209157,We (2018),Drama,We
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing


In [5]:
vectorizer = TfidfVectorizer(ngram_range = (1, 2))
tfidf = vectorizer.fit_transform(df_movie["cleaned title"])
# search engine do TF*idf
# invers Document Frequency -> how unique of particular word
def search(title):
    
    title = fn.clean_title(title)
    query_vector = vectorizer.transform([title])

    similarity = cosine_similarity(query_vector, tfidf).flatten()
    index = np.argpartition(similarity, -5)[-5:]
    return df_movie.iloc[index][::-1]

In [6]:
search("Toy Story 1995")

,movieId,title,genres,cleaned title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story
14813,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,Toy Story
3021,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,Toy Story
59767,201588,Toy Story 4 (2019),Adventure|Animation|Children|Comedy,Toy Story
20497,106022,Toy Story of Terror (2013),Animation|Children|Comedy,Toy Story of Terror
